# Readme
In this computational experiment, the aim is to verify the binding of forward and reverse primers to the target sequence. The primary objective is to identify the resulting DNA sequences and their corresponding base pair (bp) bands that would be generated in a PCR experiment. This virtual experiment is a crucial step towards performing a real experiment and assessing the availability of reagents.

The experiment revealed that a PCR product necessary for HIV research, particularly for subtype A1 and A6 prevalent in Kazakhstan, can consistently generated by pairing A with D, and in a few cases 3LTR with D. There are rare cases when 3LTR and 5LTR primers bind.

# Data

Information about HIV plasmid: https://www.hivreagentprogram.org/Catalog/Clones/ARP-194.aspx. This plasmid is circular.
Some primers contain unusual letters, such as W or R. They mean W = A or T
R = A or G. Those primers were omitted.

In [1]:
# import modules
from Bio.Seq import Seq
from Bio import SeqIO
import pandas as pd

In [2]:
# get reference sequence for HIV plasmid
with open('Sequences/hiv_plasmid.txt') as f:
    plasmid_HIV = f.read()
    
len(plasmid_HIV)
print(plasmid_HIV[:100] + '...')

CATCAATGAGGAAGCTGCAGAATGGGATAGAGTACATCCAGTGCATGCAGGGCCTATTGCACCAGGCCAGATGAGAGAACCAAGGGGAAGTGACATAGCA...


In [3]:
# create a list of primers
primers = {'A': 'CAG GAG CAG ATG ATA CAG',
           'D': 'CCC TTC ACC TTT CCA GAG',
           'LSS': 'GTCCCTTAAGCGGAG',
           'LSP': 'GTAATACGACTCACTATAGGGCCTCCGCTTAAGGGACT',
           '3LTR': 'TGTGACTCTGGTAACTAGAGATCCCTC',
           '3LTRnest': 'GAGATCCCTCAGACCCTTTTAGTCAG',
           '5LTR': 'TCAGGGAAGTAGCCTTGTGTGTGGT',
           '5LTRnest': 'CACTGTTGTCTTTTCTGGGAGTGAACTAGCC',}

# clean primers
for key in primers:
    seq = primers[key]
    primers[key] = "".join(seq.split())

primers

{'A': 'CAGGAGCAGATGATACAG',
 'D': 'CCCTTCACCTTTCCAGAG',
 'LSS': 'GTCCCTTAAGCGGAG',
 'LSP': 'GTAATACGACTCACTATAGGGCCTCCGCTTAAGGGACT',
 '3LTR': 'TGTGACTCTGGTAACTAGAGATCCCTC',
 '3LTRnest': 'GAGATCCCTCAGACCCTTTTAGTCAG',
 '5LTR': 'TCAGGGAAGTAGCCTTGTGTGTGGT',
 '5LTRnest': 'CACTGTTGTCTTTTCTGGGAGTGAACTAGCC'}

# Function to find PCR product

One way to learn about a possible product is to run this experiment from https://www.bioinformatics.org/sms2/pcr_products.html. The code below achieves the same goal.

In [4]:
# function that finds PCR product


def pcr_product(forward_primer, reverse_primer, target_seq):
    """
    find PCR product along with its indexes
    treats target molecule as linear molecule
    """
    
    # target seq
    target_seq = Seq(target_seq)
    
    # convert reverse primer
    forward_primer = Seq(forward_primer)
    reverse_primer = Seq(reverse_primer[::-1]).complement()
    
    # find indexes of primers
    forward_index = target_seq.find(forward_primer)
    reverse_index = target_seq.find(reverse_primer) + len(reverse_primer)
    
    # define length and a sequence of a product
    if forward_index != -1 and reverse_index != -1 and forward_index < reverse_index:
        product = target_seq[forward_index:reverse_index]
        return product, (forward_index, reverse_index, reverse_index-forward_index)
    else:
        return [], (forward_index, reverse_index, -1)

In [5]:
# find PCR product
PCR_result, indexes = pcr_product(primers['A'], primers['D'], plasmid_HIV)
print(len(PCR_result))
print(indexes)
print(PCR_result)

2642
(964, 3606, 2642)
CAGGAGCAGATGATACAGTATTAGAAGAAATGAGTTTGCCAGGAAGATGGAAACCAAAAATGATAGGGGGAATTGGAGGTTTTATCAAAGTAAGACAGTATGATCAGATACTCATAGAAATCTGTGGACATAAAGCTATAGGTACAGTATTAGTAGGACCTACACCTGTCAACATAATTGGAAGAAATCTGTTGACTCAGATTGGTTGCACTTTAAATTTTCCCATTAGCCCTATTGAGACTGTACCAGTAAAATTAAAGCCAGGAATGGATGGCCCAAAAGTTAAACAATGGCCATTGACAGAAGAAAAAATAAAAGCATTAGTAGAAATTTGTACAGAAATGGAAAAGGAAGGGAAAATTTCAAAAATTGGGCCTGAGAATCCATACAATACTCCAGTATTTGCCATAAAGAAAAAAGACAGTACTAAATGGAGAAAATTAGTAGATTTCAGAGAACTTAATAAGAGAACTCAAGACTTCTGGGAAGTTCAATTAGGAATACCACATCCCGCAGGGTTAAAAAAGAAAAAATCAGTAACAGTACTGGATGTGGGTGATGCATATTTTTCAGTTCCCTTAGATGAAGACTTCAGGAAGTATACTGCATTTACCATACCTAGTATAAACAATGAGACACCAGGGATTAGATATCAGTACAATGTGCTTCCACAGGGATGGAAAGGATCACCAGCAATATTCCAAAGTAGCATGACAAAAATCTTAGAGCCTTTTAAAAAACAAAATCCAGACATAGTTATCTATCAATACATGGATGATTTGTATGTAGGATCTGACTTAGAAATAGGGCAGCATAGAACAAAAATAGAGGAGCTGAGACAACATCTGTTGAGGTGGGGACTTACCACACCAGACAAAAAACATCAGAAAGAACCTCCATTCCTTTGGATGGGTTATGAACTCCATCCTGATAAATGGACAGTACAGCCTATAGTGCTGCCAGAAAAAGACAGCTGG

In [6]:
# find PCR product
PCR_result, indexes = pcr_product(primers['D'], primers['A'], plasmid_HIV)
print(len(PCR_result))
print(indexes)
print(PCR_result)

0
(-1, 17, -1)
[]


# PCR products for HIV primers on HIV plasmid

Serrao et al. in "Amplification, Next-generation Sequencing, and Genomic DNA Mapping of Retroviral Integration Sites" https://www.jove.com/t/53840/amplification-next-generation-sequencing-genomic-dna-mapping gives protocol for working with HIV virus. The protocol suggests primers for PCR amplification and linker construction. Those are LSS, 3LTR, LSP and others, see below.

The primer sequences themselves do not inherently contain information about their orientation or direction. The terms "forward" and "reverse" are used to describe the directionality of the PCR amplification, which depends on the specific application and experimental setup. That is why the code below tests "A" and "D" as well as "D" and "A".

In [7]:
count = 0

In [8]:
# run pcr with suggested primers
def run_pcr(suggested_primers, target_seq, target_name):
    '''
    Run pcr with select primers. Account for circular DNA.
    
    suggested_primers must be a list with length at least 2.
    target_seq - DNA in string format
    
    It uses primers dict defined above
    '''
    global count
    
    target_seq_rotated = target_seq[200:] + target_seq[:200]
    
    if len(primers) < 2:
        raise ValueError("Less than 2 primers")
        
    target_seq_rotate = target_seq[200:] + target_seq[:200] # rotation to the right
    
    for i in suggested_primers:
        
        for j in suggested_primers:
            # if i != j: # this line does not affect the outcome
            count += 2
            PCR_result, indexes = pcr_product(primers[i], primers[j], target_seq)
            PCR_result_rotated, indexes_rotated = pcr_product(primers[i], primers[j], target_seq_rotated)
            if indexes[2] != -1 and indexes_rotated[2] == indexes[2]:
                print(f"Forward primer {i} and reverse primer {j} produce seq of length {indexes[2]} in {target_name}")
            elif indexes[2] != -1:
                print(f"Forward primer {i} and reverse primer {j} produce seq of length {indexes[2]} in {target_name}")
            elif indexes_rotated[2] != -1:
                print(f"Forward primer {i} and reverse primer {j} produce seq of length {indexes_rotated[2]} in {target_name}")



In [9]:
# define HIV specific primers, suggested
# A and D are used like a control sample
suggested_primers = ['A','D','LSS','LSP','3LTR','3LTRnest','5LTR','5LTRnest']
target_seq = plasmid_HIV

# run pcr
run_pcr(suggested_primers, target_seq, 'HIV plasmid')
print('test for symmetrical result')
run_pcr(suggested_primers, target_seq[200:]+target_seq[:200], 'HIV plasmid')

Forward primer A and reverse primer D produce seq of length 2642 in HIV plasmid
Forward primer A and reverse primer 5LTR produce seq of length 6833 in HIV plasmid
test for symmetrical result
Forward primer A and reverse primer D produce seq of length 2642 in HIV plasmid
Forward primer A and reverse primer 5LTR produce seq of length 6833 in HIV plasmid


# PCR products for HIV primers on HXB2

HXB2 is used as a reference for HIV genome.

In [10]:
# read HXB2
HXB2 = SeqIO.read('Sequences/HXB2_reference.fasta', 'fasta')

In [11]:
# define HIV specific primers, suggested
# A and D are used like a control sample
# suggested_primers = ['A','D','LSS','LSP','3LTR','3LTRnest','5LTR','5LTRnest']
suggested_primer = primers.keys()
target_seq = str(HXB2.seq)

# run pcr
run_pcr(suggested_primers, target_seq, 'HXB2')
print('test for symmetrical result')
run_pcr(suggested_primers, target_seq[200:]+target_seq[:200], 'HXB2')

Forward primer A and reverse primer D produce seq of length 2642 in HXB2
Forward primer A and reverse primer 5LTR produce seq of length 6836 in HXB2
Forward primer 3LTR and reverse primer D produce seq of length 4397 in HXB2
Forward primer 3LTR and reverse primer 5LTR produce seq of length 8591 in HXB2
Forward primer 3LTRnest and reverse primer D produce seq of length 4380 in HXB2
Forward primer 3LTRnest and reverse primer 5LTR produce seq of length 8574 in HXB2
test for symmetrical result
Forward primer A and reverse primer D produce seq of length 2642 in HXB2
Forward primer A and reverse primer 5LTR produce seq of length 6836 in HXB2
Forward primer 3LTR and reverse primer D produce seq of length 4397 in HXB2
Forward primer 3LTR and reverse primer 5LTR produce seq of length 8591 in HXB2
Forward primer 3LTRnest and reverse primer D produce seq of length 4380 in HXB2
Forward primer 3LTRnest and reverse primer 5LTR produce seq of length 8574 in HXB2


# PCR products for HIV primers on HIV subtype A1 and A6

In Kazakhstan and other FSU countries, HIV subtype A1 and A6 are the common occuring subtypes. The data with full genomes are available in Los Alamos National Laboratory (LANL) HIV Sequence Database https://www.hiv.lanl.gov/content/sequence/NEWALIGN/align.html#subtype_6.

In [12]:
# print names of A1 and A6 subsequences
hivs = SeqIO.parse('Sequences/HIV1_REF_2020_genome_DNA.fasta', 'fasta')

for hiv in hivs:
    if hiv.name.split('.')[1] in ['A1', 'A6']:
        print(hiv.name)

Ref.A1.AU.03.PS1044_Day0.DQ676872
Ref.A1.RW.92.92RW008.AB253421
Ref.A1.UG.92.92UG037_A40.AB253429
Ref.A6.IT.02.60000.EU861977
Ref.A6.UA.00.98UA0116.AF413987
Ref.A6.UA.12.DEMA112UA024.KU749403


In [13]:
# define HIV specific primers, suggested
# A and D are used like a control sample
suggested_primers = ['A','D','LSS','LSP','3LTR','3LTRnest','5LTR','5LTRnest']

# run PCR
hivs = SeqIO.parse('Sequences/HIV1_REF_2020_genome_DNA.fasta', 'fasta')

for hiv in hivs:
    if hiv.name.split('.')[1] in ['A1', 'A6']:
        target_seq = str(hiv.seq.replace("-", ""))
        
        # run pcr
        run_pcr(suggested_primers, target_seq, hiv.name)

Forward primer A and reverse primer D produce seq of length 2642 in Ref.A1.RW.92.92RW008.AB253421
Forward primer 3LTR and reverse primer D produce seq of length 4408 in Ref.A1.RW.92.92RW008.AB253421
Forward primer A and reverse primer D produce seq of length 2642 in Ref.A1.UG.92.92UG037_A40.AB253429
Forward primer A and reverse primer 5LTR produce seq of length 6878 in Ref.A1.UG.92.92UG037_A40.AB253429
Forward primer 3LTR and reverse primer D produce seq of length 4413 in Ref.A1.UG.92.92UG037_A40.AB253429
Forward primer 3LTR and reverse primer 5LTR produce seq of length 8649 in Ref.A1.UG.92.92UG037_A40.AB253429
Forward primer A and reverse primer D produce seq of length 2642 in Ref.A6.IT.02.60000.EU861977
Forward primer A and reverse primer D produce seq of length 2642 in Ref.A6.UA.00.98UA0116.AF413987
Forward primer A and reverse primer D produce seq of length 2642 in Ref.A6.UA.12.DEMA112UA024.KU749403
Forward primer 3LTR and reverse primer D produce seq of length 4411 in Ref.A6.UA.12

# Save primers

In [14]:
# create dataframe
df = pd.DataFrame([primers], index=[0])
df.transpose()

,0
A,CAGGAGCAGATGATACAG
D,CCCTTCACCTTTCCAGAG
LSS,GTCCCTTAAGCGGAG
LSP,GTAATACGACTCACTATAGGGCCTCCGCTTAAGGGACT
3LTR,TGTGACTCTGGTAACTAGAGATCCCTC
3LTRnest,GAGATCCCTCAGACCCTTTTAGTCAG
5LTR,TCAGGGAAGTAGCCTTGTGTGTGGT
5LTRnest,CACTGTTGTCTTTTCTGGGAGTGAACTAGCC


In [15]:
# save to csv
df.to_csv('Sequences/primers.csv', index=False, header=False)

In [16]:
count

1280